# Assignment 3 : Logistic Regression

In this assignment you have to use the logistic regression technique you learned in the week 3 of the prescribed course. You have to use numpy to complete the assignment

#### Import other the libraries here

In [1]:
import numpy as np
import matplotlib.pyplot as pl
import sklearn.linear_model as skl
import sklearn as sk
import cv2 as cv
import random 

##### Define these functions here:
- Sigmoid function: `def sigmoid(x):`

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

- cost function: `def cost_fn(x,y,theta):` (refer to the course)

In [3]:
def thx(theta, x):
    temp = 0
    for i in range(0,len(x)):
        temp += theta[i]*x[i]
    return temp

def cost_fn(x, y, theta):
    sum = 0
    m=len(y)
    for i in range(m):
        sum += -(y[i]*(np.log(sigmoid(thx(theta, x[i])))) + (1 - y[i])*(np.log(1 - sigmoid(thx(theta, x[i])))))*1/m
    return sum

- Gradient descent: `def gradient_descent(X, y, theta, learning_rate=1, iters):`

$\frac{\partial J}{\partial \theta} = \frac{1}{m} \Sigma_{i=1}^m ((h_\theta(x^{(i)})-y^{(i)})x_j^{(i)})$

$\theta_{j+1}^{(i)} = \theta_j^{(i)} - \frac{1}{m} \Sigma_{i=1}^m ((h_\theta(x^{(i)})-y^{(i)})x_j^{(i)})$

Since these are for each element of your matrix. The final expression will be: (if using matrices)
$\theta_{j+1} = \theta_j - \frac{1}{m} \Sigma_{i=1}^m ((h_\theta(x)-y)x_j)$

You have to initialise $\theta$ and update it at each iteration according to this gradient descent equation

In [4]:
def gradient_descent(x, y, theta, num_iter, learning_rate = 1):
    
    a = learning_rate
    m = len(y)
    cost=np.array([])
    
    for i in range(num_iter):
        sum = np.zeros(len(theta))
        for j in range(0,m):
            sum += (1/m)*(sigmoid(thx(theta, x[j])) - y[j])*x[j]
        
        theta = theta - a*sum
        cost = np.append(cost, cost_fn(x, y, theta))
        
    return theta, cost

- Predict: `def predict(X, y, theta, learning_rate=1, iters)`:

In [6]:
def mclass(y):
    
    mdata = np.array([])
    for i in range(10):
        temp = np.array([])
        for j in range(5000):
            if(y[j] == i):
                temp = np.append(temp, 1)
            else:
                temp = np.append(temp, 0)
        
        mdata = np.append(mdata,temp)
   
    return mdata.reshape(10,5000)
    

def predict(x, y, theta, num_iter, learning_rate=1):
    
    y = mclass(y)
    mod_theta = np.array([])
    cost = np.array([])
    
    for i in range(10):
        temp_t, temp_c = gradient_descent(x, y[i], theta, num_iter, learning_rate)
        mod_theta = np.append(mod_theta, temp_t)
        cost = np.append(cost, temp_c)
    
    mod_theta = mod_theta.reshape(10,401)
    cost = cost.reshape(10, num_iter)
    
    return mod_theta, cost

def prob(x, y, theta, num_iter, learning_rate = 1):
    
    mod_theta, cost = predict(x, y, theta, num_iter, learning_rate)
    pdata = np.array([])
    
    for i in range(len(y)):
        for j in range(10):
            temp = sigmoid(thx(mod_theta[j], x[i]))
            pdata = np.append(pdata, temp)
    
    pdata_reshape = pdata.reshape(5000,10)
    prediction = np.array([])
    
    for i in range(5000):
        prediction = np.append(prediction, np.argmax(pdata_reshape[i]))
    
    return prediction, pdata_reshape, cost, mod_theta

### Load MNSIT data here

There are two file, X.csv and y.csv.
You have to load these csv files (read about csv in python) and store them in python variables. 
There are 5000 images, each line in X.csv is an image (pixels of image of size 20x20 are concatenated to size 400x1) and each line is y.csv is the label of that image (label in n-th line in y.csv corresponds to image in n-th line in X.csv)

In [7]:

#initializations
x = np.loadtxt('X.csv', delimiter = ',')
y = np.loadtxt('y.csv')
theta = np.zeros(len(x[0]) + 1)
num_iter = 10
#print(x)
#print(y)

Now your task is to:
1. Reshape these to 20x20 and show any 5 of them (random) here. 
2. For regression, you have to use the 400x1 data only. (X will be a 5000x400 matrix and y will be a 1x400 matrix)
3. Get you prediction and compare it with the labels in y
4. Calculate the error %

In [11]:
x_reshaped = (x.flatten()).reshape(5000,20,20)

a = x_reshaped[random.randint(0,4995)]
for i in range(9):
    a = np.hstack((a, x_reshaped[random.randint(0,5000)]))
cv.imshow("Random_image",a)
k=cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
e = np.zeros((5000,1))
for i in range(5000):
    e[i][0] = 1
mod_x = np.hstack((e,x))

prediction, pdata, cost, mod_theta = prob(mod_x, y, theta, num_iter, 1)

In [22]:
count = 0
for i in range(5000):
    if(prediction[i] == y[i]):
        count += 1
print("Number of accurate predictions = ", count)

error=np.array([])
for i in range(5000):
    error = np.append(error, 1 - np.max(pdata[i]))
error = error*100
print("Percent error", error)

Number of accurate predictions =  4200
Percent error [ 8.28709531 11.76497245 91.69788505 ... 65.78423401 76.35598146
 82.90295562]


Generate the confusion matrix here and show it here:



In [23]:
print("Confusion Matrix: ", '\n')
print(sk.metrics.confusion_matrix(y,prediction,labels=[0,1,2,3,4,5,6,7,8,9]))

Confusion Matrix:  

[[476   1   1   2   1   0  13   0   6   0]
 [  0 481   1   1   0   2   2   0  13   0]
 [ 10  19 397  15   9   1  18   6  22   3]
 [  3   7  19 411   0  13   4   7  27   9]
 [  2   7   5   1 427   0  14   3   8  33]
 [ 19  15   7  72  17 305  16   1  34  14]
 [  9  13  10   0   4   3 457   1   3   0]
 [ 10  21   6   1  13   0   0 434   1  14]
 [  2  11  10  27   3   7   7   2 419  12]
 [ 10   7   5   7  34   5   2  25  12 393]]


#### Bonus task: 
Use [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to perform logistic regression (just a single function which will handle all your task :P)

In [25]:
regression = skl.LogisticRegression().fit(x,y)
sk_prediction=log_reg.predict(x)

c:\users\bhavy\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
